In [1]:
# set cwd and solver
import os
import cobra
cobra.Configuration().solver = "gurobi"
os.chdir('C:/Users/prins/GitHub/Human1_RPE-PR') 

# load all models in \cs_mods\July2023 folder by cycling through names
from cobra.io import read_sbml_model    
import glob 
import os   
from pathlib import Path
import pandas as pd

# do not change directory but still go to the folder with the models
folder = Path().cwd() / "cs_mods" / "July2023"
model_list = []
for file in folder.glob("*.xml"):  
    model = read_sbml_model(file)
    model.id = file.stem # remove .xml 
    model.name = file.stem # remove .xml
    model_list.append(model)

# load old combined RPE_PR models
mod_RPE_PR = read_sbml_model(Path().cwd() / "models" / "mod_RPE_PR.xml")
mod_RPE_PR.id = 'RPE_PR_old'
mod_RPE_PR.name = 'RPE_PR_old'
mod_Human1_Human1 = read_sbml_model(Path().cwd() / "models" /  "mod_Human1_Human1.xml")
mod_Human1_Human1.id = 'RPE_PR_control_old'
mod_Human1_Human1.name = 'RPE_PR_control_old'
mod = read_sbml_model(Path().cwd() / 'models/Human-GEM.xml')
mod.id = 'Human1_old'
mod.name = 'Human1_old'

# add old models to model_list
model_list = model_list + [mod,mod_RPE_PR]

# check out model_list
model_list

Set parameter Username
Academic license - for non-commercial use only - expires 2024-02-03


No objective coefficients in model. Unclear what should be optimized
No objective coefficients in model. Unclear what should be optimized
No objective coefficients in model. Unclear what should be optimized
No objective coefficients in model. Unclear what should be optimized


[<Model Human-GEM_28062023 at 0x296a913e9b0>,
 <Model mod_RPE_PRcone_Liang at 0x296f310f430>,
 <Model mod_RPE_PRcone_Liang_singlePR at 0x296fc346da0>,
 <Model mod_RPE_PRcone_Liang_singleRPE at 0x296fc793670>,
 <Model mod_RPE_PRcone_Lukowski at 0x2968f3d2cb0>,
 <Model mod_RPE_PRcone_Lukowski_singlePR at 0x29699122680>,
 <Model mod_RPE_PRcone_Lukowski_singleRPE at 0x29698a58280>,
 <Model mod_RPE_PRcontrol at 0x29703042ce0>,
 <Model mod_RPE_PRrod_Liang at 0x297149d7b20>,
 <Model mod_RPE_PRrod_Liang_singlePR at 0x2975e86e980>,
 <Model mod_RPE_PRrod_Liang_singleRPE at 0x297625f3a90>,
 <Model mod_RPE_PRrod_Lukowski at 0x29766716440>,
 <Model mod_RPE_PRrod_Lukowski_singlePR at 0x2976e14add0>,
 <Model mod_RPE_PRrod_Lukowski_singleRPE at 0x29772087d60>,
 <Model Human1_old at 0x29792eafb80>,
 <Model RPE_PR_old at 0x29718ddfcd0>]

In [41]:
model_list[0].reactions.get_by_id('MAR09048')

Reaction identifier,MAR09048
Name,
Memory address,0x296af9a6a40
Stoichiometry,MAM02630e <=> O2 <=>
GPR,
Lower bound,-1000
Upper bound,1000


In [42]:
# modify models
# load blood exchange bounds

# for all models in the list set objective function (oxygen consumption, in RPE for combined models)
for m in model_list:
    m.objective_direction = 'max' # maximizing this reaction will minimize oxygen uptake (due to direction of reaction)
    if 'MAR09048_RPE' in [r.id for r in m.reactions]:
        m.objective = 'MAR09048_RPE'
    elif 'MAR09048_PR' in [r.id for r in m.reactions]:
        m.objective = 'MAR09048_PR'
    elif 'MAR09048_eRPE_PR' in [r.id for r in m.reactions]:
        m.objective = 'MAR09048_eRPE_PR'
    elif 'MAR09048' in [r.id for r in m.reactions]:
        m.objective = 'MAR09048'
    else:
        print('no oxygen exchange reaction (MAR09048) in model: ' + m.id)

In [43]:
model_list[0].reactions.get_by_id('MAR03964')

Reaction identifier,MAR03964
Name,ATP phosphohydrolase
Memory address,0x296ae80a9e0
Stoichiometry,MAM01371c + MAM02040c --> MAM01285c + MAM02039c + MAM02751c ATP + H2O --> ADP + H+ + Pi
GPR,
Lower bound,26.57
Upper bound,26.57


In [45]:
# list combined and single models
combined_models = [m for m in model_list if 'single' not in m.id and 'Human' not in m.id]
single_models = [m for m in model_list if 'single' in m.id]
Human1_models = [m for m in model_list if 'Human' in m.id]

# close PR exchange in combined models and open RPE exchange (efflux)
for m in combined_models:
    for r in [r for r in m.reactions if len(r.products) == 0 ]:           # close all exchange reactions
        r.bounds=(0,0)
    for r in [r for r in m.reactions if len(r.products) == 0 if '_RPE' in r.id]:           # open efflux for RPE exchange reactions
        r.bounds=(0,1000) 

# only allow efflux exchange in single models
for m in single_models:
    for r in [r for r in m.reactions if len(r.products) == 0 ]:           # close all exchange reactions
        r.bounds=(0,1000)

# for Human1 models, open efflux
for m in [m for m in model_list if 'Human' in m.id]:
    for r in [r for r in m.reactions if len(r.products) == 0]:           # open efflux for RPE exchange reactions
        r.bounds=(0,1000)

In [53]:
# set ATP hydrolysis  (in PR for combined models)

# RODS
rods_ATP_flux_day = 21.92
rods_ATP_flux_night = 26.57

# CONES
cones_ATP_flux_day = 73.89
cones_ATP_flux_night = 44.50

def set_ATP_flux(model_list, x):
    # for all models in the list set objective function (ATP hydrolysis, in PR for combined models)
    for m in model_list:
        if 'MAR03964_PR' in [r.id for r in m.reactions]:
            m.reactions.get_by_id('MAR03964_PR').bounds = (x,x)
        elif 'MAR03964_RPE' in [r.id for r in m.reactions]:
            m.reactions.get_by_id('MAR03964_RPE').bounds = (x,x)
        elif 'MAR03964' in [r.id for r in m.reactions]:
            m.reactions.get_by_id('MAR03964').bounds = (x,x)
        else:
            print('no ATP hydrolysis reaction (MAR03964) in model: ' + m.id)

In [81]:
ex_bounds_dark = {'MAR09048':(-1000,1000),'MAR09034':(-4.18,1000),'MAR09135':(7.25,1000)}
ex_bounds_light = {'MAR09048':(-1000,1000),'MAR09034':(-2.61,1000),'MAR09135':(2.95,1000)}

# MAR09048: oxygen
# MAR09034: glucose
# MAR09135: lactate

# DARK
# 3.51 (ox)
# 4.18 (gluc)
# 7.25 (lac)

# LIGHT
# 2.61 (ox)
# 1.93 (gluc)
# 2.95 (lac)

import warnings
from src.modify_model import set_exchange_bounds 

# set exchange bounds
set_ATP_flux(model_list, rods_ATP_flux_day)
for m in model_list:
    set_exchange_bounds(m, ex_bounds_light)
    m.optimize()
    results_dict[m.id] = m.objective.value
    warnings.filterwarnings('ignore')
df1 = pd.DataFrame(results_dict, index = ['LIGHT RODS: min oxygen uptake (pmol/s/mm^2)']).T
# set exchange bounds
set_ATP_flux(model_list, rods_ATP_flux_night)
for m in model_list:
    set_exchange_bounds(m, ex_bounds_dark)
    m.optimize()
    results_dict[m.id] = m.objective.value
    warnings.filterwarnings('ignore')
df2 = pd.DataFrame(results_dict, index = ['DARK RODS: min oxygen uptake (pmol/s/mm^2)']).T
# set exchange bounds
set_ATP_flux(model_list, cones_ATP_flux_day)
for m in model_list:
    set_exchange_bounds(m, ex_bounds_light)
    m.optimize()
    results_dict[m.id] = m.objective.value
    warnings.filterwarnings('ignore')
df3 = pd.DataFrame(results_dict, index = ['LIGHT CONES: min oxygen uptake (pmol/s/mm^2)']).T
# set exchange bounds
set_ATP_flux(model_list, cones_ATP_flux_night)
for m in model_list:
    set_exchange_bounds(m, ex_bounds_dark)
    m.optimize()
    results_dict[m.id] = m.objective.value
    warnings.filterwarnings('ignore')
df4 = pd.DataFrame(results_dict, index = ['DARK CONES: min oxygen uptake (pmol/s/mm^2)']).T
# combine dataframes
df = pd.concat([df1,df2,df3,df4],axis=1)
df

,LIGHT RODS: min oxygen uptake (pmol/s/mm^2),DARK RODS: min oxygen uptake (pmol/s/mm^2),LIGHT CONES: min oxygen uptake (pmol/s/mm^2),DARK CONES: min oxygen uptake (pmol/s/mm^2)
Human-GEM_28062023,-3.367436,None,None,None
mod_RPE_PRcone_Liang,-3.458889,None,None,None
mod_RPE_PRcone_Liang_singlePR,None,None,None,None
mod_RPE_PRcone_Liang_singleRPE,-3.458889,None,None,None
mod_RPE_PRcone_Lukowski,-3.367436,None,None,None
mod_RPE_PRcone_Lukowski_singlePR,-3.696667,None,None,None
mod_RPE_PRcone_Lukowski_singleRPE,-3.458889,None,None,None
mod_RPE_PRcontrol,-3.367436,None,None,None
mod_RPE_PRrod_Liang,-3.458889,None,None,None
mod_RPE_PRrod_Liang_singlePR,None,None,None,None


In [79]:
###### RELAX LACTATE CONSTRAINTS ######

ex_bounds_dark = {'MAR09048':(-1000,1000),'MAR09034':(-4.18,1000),'MAR09135':(0,1000)}
ex_bounds_light = {'MAR09048':(-1000,1000),'MAR09034':(-2.61,1000),'MAR09135':(0,1000)}

# MAR09048: oxygen
# MAR09034: glucose
# MAR09135: lactate

# DARK
# 3.51 (ox)
# 4.18 (gluc)
# 7.25 (lac)

# LIGHT
# 2.61 (ox)
# 1.93 (gluc)
# 2.95 (lac)

import warnings
from src.modify_model import set_exchange_bounds 

# set exchange bounds
set_ATP_flux(model_list, rods_ATP_flux_day)
for m in model_list:
    set_exchange_bounds(m, ex_bounds_light)
    m.optimize()
    results_dict[m.id] = m.objective.value
    warnings.filterwarnings('ignore')
df1 = pd.DataFrame(results_dict, index = ['LIGHT RODS: min oxygen uptake (pmol/s/mm^2)']).T
# set exchange bounds
set_ATP_flux(model_list, rods_ATP_flux_night)
for m in model_list:
    set_exchange_bounds(m, ex_bounds_dark)
    m.optimize()
    results_dict[m.id] = m.objective.value
    warnings.filterwarnings('ignore')
df2 = pd.DataFrame(results_dict, index = ['DARK RODS: min oxygen uptake (pmol/s/mm^2)']).T
# set exchange bounds
set_ATP_flux(model_list, cones_ATP_flux_day)
for m in model_list:
    set_exchange_bounds(m, ex_bounds_light)
    m.optimize()
    results_dict[m.id] = m.objective.value
    warnings.filterwarnings('ignore')
df3 = pd.DataFrame(results_dict, index = ['LIGHT CONES: min oxygen uptake (pmol/s/mm^2)']).T
# set exchange bounds
set_ATP_flux(model_list, cones_ATP_flux_night)
for m in model_list:
    set_exchange_bounds(m, ex_bounds_dark)
    m.optimize()
    results_dict[m.id] = m.objective.value
    warnings.filterwarnings('ignore')
df4 = pd.DataFrame(results_dict, index = ['DARK CONES: min oxygen uptake (pmol/s/mm^2)']).T
# combine dataframes
df = pd.concat([df1,df2,df3,df4],axis=1)
df

,LIGHT RODS: min oxygen uptake (pmol/s/mm^2),DARK RODS: min oxygen uptake (pmol/s/mm^2),LIGHT CONES: min oxygen uptake (pmol/s/mm^2),DARK CONES: min oxygen uptake (pmol/s/mm^2)
Human-GEM_28062023,-3.34,-3.64200,-13.952308,-7.228
mod_RPE_PRcone_Liang,-3.34,-3.64200,-14.801143,-7.228
mod_RPE_PRcone_Liang_singlePR,None,-3.64200,None,None
mod_RPE_PRcone_Liang_singleRPE,-3.34,-3.64200,-15.21,-7.228
mod_RPE_PRcone_Lukowski,-3.34,-3.64200,-13.952308,-7.228
mod_RPE_PRcone_Lukowski_singlePR,-3.34,-3.64200,None,-7.228
mod_RPE_PRcone_Lukowski_singleRPE,-3.34,-3.64200,-15.21,-7.228
mod_RPE_PRcontrol,-3.34,-3.64200,-13.952308,-7.228
mod_RPE_PRrod_Liang,-3.34,-3.64200,-14.801143,-7.228
mod_RPE_PRrod_Liang_singlePR,-3.34,-3.64200,None,-7.228


In [82]:
###### MORE GLUCOSE AVAILABLE (8 pmol/s/mm^2) ######

ex_bounds_dark = {'MAR09048':(-1000,1000),'MAR09034':(-8,1000),'MAR09135':(7.25 ,1000)}
ex_bounds_light = {'MAR09048':(-1000,1000),'MAR09034':(-8,1000),'MAR09135':(2.95,1000)}

# MAR09048: oxygen
# MAR09034: glucose
# MAR09135: lactate

# DARK
# 3.51 (ox)
# 4.18 (gluc)
# 7.25 (lac)

# LIGHT
# 2.61 (ox)
# 1.93 (gluc)
# 2.95 (lac)

import warnings
from src.modify_model import set_exchange_bounds 

# set exchange bounds
set_ATP_flux(model_list, rods_ATP_flux_day)
for m in model_list:
    set_exchange_bounds(m, ex_bounds_light)
    m.optimize()
    results_dict[m.id] = m.objective.value
    warnings.filterwarnings('ignore')
df1 = pd.DataFrame(results_dict, index = ['LIGHT RODS: min oxygen uptake (pmol/s/mm^2)']).T
# set exchange bounds
set_ATP_flux(model_list, rods_ATP_flux_night)
for m in model_list:
    set_exchange_bounds(m, ex_bounds_dark)
    m.optimize()
    results_dict[m.id] = m.objective.value
    warnings.filterwarnings('ignore')
df2 = pd.DataFrame(results_dict, index = ['DARK RODS: min oxygen uptake (pmol/s/mm^2)']).T
# set exchange bounds
set_ATP_flux(model_list, cones_ATP_flux_day)
for m in model_list:
    set_exchange_bounds(m, ex_bounds_light)
    m.optimize()
    results_dict[m.id] = m.objective.value
    warnings.filterwarnings('ignore')
df3 = pd.DataFrame(results_dict, index = ['LIGHT CONES: min oxygen uptake (pmol/s/mm^2)']).T
# set exchange bounds
set_ATP_flux(model_list, cones_ATP_flux_night)
for m in model_list:
    set_exchange_bounds(m, ex_bounds_dark)
    m.optimize()
    results_dict[m.id] = m.objective.value
    warnings.filterwarnings('ignore')
df4 = pd.DataFrame(results_dict, index = ['DARK CONES: min oxygen uptake (pmol/s/mm^2)']).T
# combine dataframes
df = pd.concat([df1,df2,df3,df4],axis=1)
df

,LIGHT RODS: min oxygen uptake (pmol/s/mm^2),DARK RODS: min oxygen uptake (pmol/s/mm^2),LIGHT CONES: min oxygen uptake (pmol/s/mm^2),DARK CONES: min oxygen uptake (pmol/s/mm^2)
Human-GEM_28062023,-1.18400,-2.11400,-11.578,-5.7
mod_RPE_PRcone_Liang,-1.18400,-2.11400,-11.578,-5.7
mod_RPE_PRcone_Liang_singlePR,-1.18400,-2.11400,None,None
mod_RPE_PRcone_Liang_singleRPE,-1.18400,-2.11400,-11.578,-5.7
mod_RPE_PRcone_Lukowski,-1.18400,-2.11400,-11.578,-5.7
mod_RPE_PRcone_Lukowski_singlePR,-1.18400,-2.11400,-11.578,-5.7
mod_RPE_PRcone_Lukowski_singleRPE,-1.18400,-2.11400,-11.578,-5.7
mod_RPE_PRcontrol,-1.18400,-2.11400,-11.578,-5.7
mod_RPE_PRrod_Liang,-1.18400,-2.11400,-11.578,-5.7
mod_RPE_PRrod_Liang_singlePR,-1.18400,-2.11400,-11.578,-5.7
